In [1]:
print("OK")

OK


In [2]:
import os
print(os.getcwd())


c:\Users\WELCOME\Desktop\Medical-Chatbot\research


In [3]:
%pwd

'c:\\Users\\WELCOME\\Desktop\\Medical-Chatbot\\research'

In [5]:
import os
os.chdir("../")

In [6]:
import os

# Change working directory to your project folder
os.chdir("C:/Users/WELCOME/Desktop/Medical-Chatbot")
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\WELCOME\Desktop\Medical-Chatbot


In [7]:
%pwd

'C:\\Users\\WELCOME\\Desktop\\Medical-Chatbot'

In [8]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

c:\Users\WELCOME\miniconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
#Extract Data From the PDF File
#Tabnine: Edit|Test|Explain|Document|Ask
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents= loader.load()

    return documents

In [12]:
import os
print(os.getcwd())


C:\Users\WELCOME\Desktop\Medical-Chatbot


In [13]:
extracted_data=load_pdf_file(data='Data/')

In [67]:
#extracted_data

In [14]:
#split the data into text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [15]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

In [17]:
#download the embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [18]:
embeddings = download_hugging_face_embeddings()

In [19]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [54]:
#query_result

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')


In [23]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-vqijzyf.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [24]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY


In [25]:
#embed each chunk and upsert the embeddings into your Pinecone index.

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
) 

c:\Users\WELCOME\miniconda3\envs\medibot\lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [26]:
#load existing index

from langchain_pinecone import PineconeVectorStore
#embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [27]:
docsearch

In [28]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [29]:
retriever_docs = retriever.invoke("What is Acne?")

In [30]:
retriever_docs

[Document(id='e3c65ef4-3b34-4994-b2ac-cd5b5dd37c2b', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='5dde1c70-cac3-4487-95f6-9b2abde146c2', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nat

In [31]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.1", temperature=0.4)


In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [34]:
response = rag_chain.invoke({"input": "What is Acromegaly and gigantism?"})
print(response['answer'])


Acromegaly is a disorder caused by the abnormal release of growth hormone from the pituitary gland, leading to increased growth in bone and soft tissue. It is characterized by unusual height and various other bodily disturbances. When this abnormality occurs after bone growth stops, it is specifically referred to as acromegaly.
